Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

- Наименование вакансии.
- Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
- Ссылку на саму вакансию.
- Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [1]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup as bs

In [81]:
def parse_hh(vacancy):
    vacancy_data = {}
    names, min_salaries, max_salaries, currencies, urls, cites = [], [], [], [], [], []
    params = {
        'text': vacancy,
        'search_field': 'name',
        'items_on_page': '100',
        'page': ''
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
    }
    url = 'https://hh.ru/search/vacancy'
    response = requests.get(url, params=params, headers=headers)
    dom = bs(response.text, 'html.parser')
    pages_data = dom.find('div', {'class': 'bloko-gap bloko-gap_top'}).findChild()
    last_page = int(pages_data.find_all('span')[-3].getText())
    for page in range(0, last_page):
        params['page'] = page
        response= requests.get(url, params=params, headers=headers)
        if response.ok:
            dom = bs(response.text, 'html.parser')
            vacancy_items_data = dom.find('div', {'data-qa': 'vacancy-serp__results'})
            if vacancy_items_data != None:
                vacancy_items = vacancy_items_data.find_all('div', {'class': 'serp-item'})
            for vacancy_item in vacancy_items:
                vacancy_main_data = vacancy_item.find('a', {'class': 'serp-item__title'})
                names.append(vacancy_main_data.getText())
                urls.append(vacancy_main_data['href'])
                cites.append('/'.join(vacancy_main_data['href'].split('/')[:3]))
                vacancy_salary_data = vacancy_item.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
                min_salary, max_salary, currency = None, None, None
                if vacancy_salary_data != None:
                    vacancy_salary = vacancy_salary_data.getText()
                    salary_list = re.findall('[0-9]+', vacancy_salary)
                    currency = vacancy_salary.split(' ')[-1]
                    if len(salary_list) == 2:
                        if 'от' in vacancy_salary:
                            min_salary = float(''.join(salary_list))
                        if 'до' in vacancy_salary:
                            max_salary = float(''.join(salary_list))
                    if len(salary_list) == 4:
                        min_salary = float(''.join(salary_list[:2]))
                        max_salary = float(''.join(salary_list[2:]))
                min_salaries.append(min_salary)
                max_salaries.append(max_salary)
                currencies.append(currency)
    vacancy_data = {'name': names, 'min_salary': min_salaries, 'max_salary': max_salaries, 'currency': currencies, 'url': urls, 'cite': cites}
    return pd.DataFrame(vacancy_data)

In [82]:
vacancy = input('Вакансия: ') # python
vacancy_dataframe = parse_hh(vacancy)
vacancy_dataframe.to_csv(f'{vacancy}.csv', index=False, encoding='utf-8', sep=',')

In [83]:
vacancy_dataframe.head(10)

,name,min_salary,max_salary,currency,url,cite
0,Backend программист (Python),120000.0,NaN,руб.,https://novosibirsk.hh.ru/vacancy/77147428?fro...,https://novosibirsk.hh.ru
1,Ведущий инженер по тестированию (python),NaN,NaN,None,https://novosibirsk.hh.ru/vacancy/73199414?fro...,https://novosibirsk.hh.ru
2,"Python Developer (проект ""Маркетинг "") Middle",NaN,NaN,None,https://novosibirsk.hh.ru/vacancy/76968960?fro...,https://novosibirsk.hh.ru
3,"Python Developer (проект ""Маркетинг БКП"") Middle",NaN,NaN,None,https://novosibirsk.hh.ru/vacancy/76978400?fro...,https://novosibirsk.hh.ru
4,"Python Developer (проект ""Маркетинг"") Senior",NaN,NaN,None,https://novosibirsk.hh.ru/vacancy/76976985?fro...,https://novosibirsk.hh.ru
5,Junior Python Backend Developer,NaN,60000.0,руб.,https://novosibirsk.hh.ru/vacancy/77236116?fro...,https://novosibirsk.hh.ru
6,Разработчик Python,200000.0,250000.0,руб.,https://novosibirsk.hh.ru/vacancy/77091465?fro...,https://novosibirsk.hh.ru
7,Разработчик Python,200000.0,250000.0,руб.,https://novosibirsk.hh.ru/vacancy/77091466?fro...,https://novosibirsk.hh.ru
8,Программист Python Junior,NaN,NaN,None,https://novosibirsk.hh.ru/vacancy/77146976?fro...,https://novosibirsk.hh.ru
9,"Python-разработчик (Django, DRF)",60000.0,220000.0,руб.,https://novosibirsk.hh.ru/vacancy/70705466?fro...,https://novosibirsk.hh.ru
